In [1]:
import os
os.chdir('../..')
import sys
sys.path.append("/workspace/src")

## Imports

In [6]:
from util.misc import load, save
from databases.datasets import PersonStackedMuPoTsDataset, PersonStackedMucoTempDataset, Mpi3dTestDataset, Mpi3dTrainDataset
from databases.joint_sets import CocoExJoints, MuPoTSJoints, JointSet
from util.pose import _calc_limb_length
from databases import mpii_3dhp, mupots_3d
import numpy as np

%matplotlib notebook

In [4]:
def plfig(lines=1, full_width=False):
    if lines==1 and not full_width:
        plt.figure()
    elif full_width:
        plt.figure(figsize=(9.5, 3*lines))
    else:
        plt.figure(figsize=(4.5, 3*lines))    

## Bone stability
Visualize the length of a bone in training/test set.

It seems in Muco-Temp (and therefore in MPII-3DHP), bones are normalized except for:  
left/right hip-knee (30-40)  
spine-neck (~10)  
neck-shoulder (5-10)

hip is also the average of lefthip/right, which is not the same as in the captury videos. It seems left/right hip joints are synthetic and are not coming directly from captury.

In [7]:
class FullMpiiSet(JointSet):
    NAMES = np.array(['spine3', 'spine4', 'spine2', 'spine', 'pelvis',     
        'neck', 'head', 'head_top', 'left_clavicle', 'left_shoulder', 'left_elbow', 
       'left_wrist', 'left_hand',  'right_clavicle', 'right_shoulder', 'right_elbow', 'right_wrist',
       'right_hand', 'left_hip', 'left_knee', 'left_ankle', 'left_foot', 'left_toe',  
       'right_hip' , 'right_knee', 'right_ankle', 'right_foot', 'right_toe'])
    
    NUM_JOINTS=28
    NAMES.flags.writeable = False

In [8]:
full_names =  ['spine3', 'spine4', 'spine2', 'spine', 'pelvis',     
        'neck', 'head', 'head_top', 'left_clavicle', 'left_shoulder', 'left_elbow', 
       'left_wrist', 'left_hand',  'right_clavicle', 'right_shoulder', 'right_elbow', 'right_wrist',
       'right_hand', 'left_hip', 'left_knee', 'left_ankle', 'left_foot', 'left_toe',  
       'right_hip' , 'right_knee', 'right_ankle', 'right_foot', 'right_toe']
parent1 = np.array([3, 1, 4, 5, 5, 2, 6, 7, 6, 9, 10, 11, 12, 6, 14, 15, 16, 17, 5, 19, 20, 21, 22, 5, 24, 25, 26, 27 ])-1
parent2 = np.array([4, 3, 5, 5, 5, 1, 2, 6, 2, 6, 9, 10, 11, 2, 6, 14, 15, 16, 4, 5, 19, 20, 21, 4, 5, 24, 25, 26]) -1

for i in range(len(full_names)):
    print('%15s %15s %s' % (full_names[i], full_names[parent1[i]], full_names[parent2[i]]))

         spine3          spine2 spine
         spine4          spine3 spine2
         spine2           spine pelvis
          spine          pelvis pelvis
         pelvis          pelvis pelvis
           neck          spine4 spine3
           head            neck spine4
       head_top            head neck
  left_clavicle            neck spine4
  left_shoulder   left_clavicle neck
     left_elbow   left_shoulder left_clavicle
     left_wrist      left_elbow left_shoulder
      left_hand      left_wrist left_elbow
 right_clavicle            neck spine4
 right_shoulder  right_clavicle neck
    right_elbow  right_shoulder right_clavicle
    right_wrist     right_elbow right_shoulder
     right_hand     right_wrist right_elbow
       left_hip          pelvis spine
      left_knee        left_hip pelvis
     left_ankle       left_knee left_hip
      left_foot      left_ankle left_knee
       left_toe       left_foot left_ankle
      right_hip          pelvis spine
     right_knee       rig

In [10]:
val_data = PersonStackedMucoTempDataset('hrnet', 'normal')
test_data = PersonStackedMuPoTsDataset('hrnet', 'normal', 'all')
refine_results = load('results_smoothed_83-2955.pkl')

FileNotFoundError: [Errno 2] No such file or directory: 'results_smoothed_83-2955.pkl'

In [ ]:
class RefResults:
    index= refine_results['index']
    poses3d= refine_results['pred']
refine_data = RefResults()

In [ ]:
mpi_train_data = Mpi3dTrainDataset('hrnet', 'normal', 'megadepth_at_hrnet', True, 2)
mpi_test_data = Mpi3dTestDataset('hrnet', 'normal', 'megadepth_at_hrnet')

In [ ]:
gt = mupots_3d.load_gt_annotations(16)
validFrame = gt['isValidFrame']

In [ ]:
BONES = [['left_ankle', 'left_knee'], ['left_hip', 'left_knee'], ['left_hip', 'hip'], 
         ['hip', 'spine'], ['spine', 'head/nose'], ['left_shoulder', 'left_elbow']]
# BONES = [['right_ankle', 'right_knee'], ['right_hip', 'right_knee'], ['right_hip', 'hip'], 
#          ['hip', 'spine'], ['spine', 'neck'], ['right_shoulder', 'right_elbow']]
# BONES = [['neck', 'right_shoulder'], ['right_shoulder', 'right_elbow'], ['right_elbow', 'right_wrist']]
# BONES = [['spine', 'neck'], ['neck', 'head/nose'], ['head/nose', 'head_top']]
# BONES=[['left_ankle', 'left_knee'], ['left_hip', 'left_knee'],['left_shoulder', 'left_elbow']]
joint_set = MuPoTSJoints()
data = mpi_train_data

seqs = np.unique(data.index.seq)
seq = np.random.choice(seqs)
# seq='16/2'
print(seq)

inds = data.index.seq==seq
plfig(1, False)
# plt.subplot(1,3,1)

names=['ankle-knee', 'knee-hip', 'elbow-shoulder']
for i,bone in enumerate(BONES):
    lens = _calc_limb_length(data.poses3d[inds], joint_set, [bone])
    plt.plot(lens, label=bone[0])
    print(np.std(lens))
# ax2=plt.gca().twinx()
# ax2.plot(gt['occlusions'][:,2, joint_set.index_of('left_shoulder')], color='black')

plt.legend()

mupots: 16/2 - jumps, all frames are valid

In [ ]:
# Mupots gt vs pred

joint_set = MuPoTSJoints()

seqs = np.unique(test_data.index.seq)
seq = np.random.choice(seqs)
print(seq)

inds = test_data.index.seq==seq
assert np.all(refine_data.index.seq[inds]==seq)

bones = [['left_ankle', 'left_knee'], ['left_knee', 'left_hip', ], ['left_hip', 'hip'],
        ['right_wrist', 'right_elbow'], ['right_elbow', 'right_shoulder', ], ['right_shoulder', 'neck']]
plfig(2, True)
for i, bone in enumerate(bones):
    plt.subplot(2,3,i+1)
    lens = _calc_limb_length(test_data.poses3d[inds], joint_set, [bone])
    plt.plot(lens, label='gt')
    lens = _calc_limb_length(refine_data.poses3d[inds], joint_set, [bone])
    plt.plot(lens, label='pred')
    plt.title('%s %s' % (bone[0], bone[1]))
plt.tight_layout()

### Mpii-train data full joints

In [ ]:
sub=7
seq=1
annot = load(os.path.join(mpii_3dhp.MPII_3DHP_PATH, 'S%d' % sub, 'Seq%d' % seq, 'annot.mat'))
annot3 = list([x[0].reshape((-1, 28, 3)).astype('float32') for x in annot['annot3']])

In [ ]:
lhip = joint_set.index_of('left_hip')
rhip = joint_set.index_of('right_hip')
np.std(np.linalg.norm((p[:,rhip]+p[:,lhip])/2-p[:,joint_set.index_of('pelvis')], axis=1))

In [ ]:
# is there any joint the knee is normalized to? answer: No
lhip = joint_set.index_of('left_knee')
p = annot3[7]
for i in range(28):
    print(i, FullMpiiSet.NAMES[i], np.std(np.linalg.norm(p[:,lhip]-p[:,i], axis=1)))

In [ ]:
# 
BONES = [['left_hip', 'pelvis'], ['neck', 'left_clavicle'], ['spine4', 'neck']]
joint_set = FullMpiiSet()

plfig()
for bone in BONES:
    lens = _calc_limb_length(annot3[9], joint_set, [bone])
    plt.plot(lens, label=bone[0])
    print(np.std(lens))
plt.legend()